# Лабораторная работа #2

Этот корпус содержит сообщения, собранные в социальной сети Twitter, автоматически размеченный на два класса (положительные и отрицательные). 
Каждый текст в корпусе имеет следующие атрибуты:

0. id
1. дата публикации;
2. имя автора;
3. текст твита;
4. класс, к которому принадлежит текст (положительный, отрицательный, нейтральный);
5. количество добавлений сообщения в избранное;
6. количество ретвитов (количество копирований этого сообщения другими пользователями);
7. количество друзей пользователя;
8. количество пользователей, у которых данный юзер в друзьях (количество фоловеров);
9. количество листов, в которых состоит пользователь.


Целью лабораторной работы является научиться применять методы обучения с учителем (supervised learning) для задачи анализа тональностей, а также освоить специализированный инструментарий для компьютерной лингвистики и машинного обучения.

1. Загрузить с использованием csv файлов твиты.
2. Провести предобработку корпуса (удаление стоп-слов, и т.д.)
3. Используя выбранную модель репрезентации получить матричное представление.
4. Используя изученные классификаторы получить численные оценки точности, полноты и F-меры.
5. Объяснить полученные результаты.

**Для начала, имортируем все необходимые библиотеки**

In [1]:
import pandas as pd
import numpy as np
import preprocessor as p
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

**Загрузка файлов, конкатенация таблиц**

In [2]:
df_pos = pd.read_csv('positive.csv', sep=';', header=None, names=["id",
                                                              "tdate", 
                                                              "tname", 
                                                              "ttext", 
                                                              "ttype", 
                                                              "trep",
                                                              "trtw",
                                                              "tfav", 
                                                              "tstcount", 
                                                              "tfoll", 
                                                              "tfrien", 
                                                              "listcount"])
df_pos['class'] = 1

In [3]:
df_neg = pd.read_csv('negative.csv', sep=';', header=None, names=["id",
                                                              "tdate", 
                                                              "tname", 
                                                              "ttext", 
                                                              "ttype", 
                                                              "trep",
                                                              "trtw",
                                                              "tfav", 
                                                              "tstcount", 
                                                              "tfoll", 
                                                              "tfrien", 
                                                              "listcount"])
df_neg['class'] = -1

Да, можно было бы размешать один раз, но нам захотелось размешать данные дважды и сначала мы отдельно мешаем позитивные и негативные твиты и потом, просто потому что нам так захотелось, размешаем еще раз уже сконкатенированный корпус. *Хотя можно было бы сделать просто последнее без последствий при условии, что мы работаем со всей выборкой, а не с её частью.* Но мы, гуманитарии, не ищем легких путей! :)

In [4]:
df_pos = df_pos.sample(frac=1) # мешаем позитивные
df_neg = df_neg.sample(frac=1) # мешаем негативные
res_df = pd.concat([df_pos[:2000], df_neg[:2000]]) # соединяем
res_df = res_df.sample(frac=1)  # мешаем еще раз, чтобы не было сначала позитивных, и затем негативных

In [ ]:
p.set_options(p.OPT.URL, p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.RESERVED) 
m = Mystem()
X = pd.Series([''.join(m.lemmatize(p.clean(i))) for i in res_df['ttext'].values])
y = res_df['class'].values

*Следующие две функции обозначим для удобства (в дальнейшем)*

In [ ]:
def evaluate(y_test, y_test_predict):
    print("Accuracy score: {0}".format(accuracy_score(y_test, y_test_predict)))
    print("Precision: {0}".format(precision_score(y_test, y_test_predict)))
    print("Recall: {0}".format(recall_score(y_test, y_test_predict)))
    print("F1-measure: {0}".format(f1_score(y_test, y_test_predict)))

In [ ]:
def vectorize():
    # Параметры max_df и min_df задают удаление стоп-слов именно для данного корпуса (пруф в документации scikit-learn)
    current_vectorizer = TfidfVectorizer(max_df=0.8, 
                                 min_df=0, 
                                 use_idf=True)
    return current_vectorizer

**Naive Bayes**

In [ ]:
kf = KFold(n_splits=3,shuffle=True) # Делим натрое
for train_index, test_index in kf.split(X):
    # Создаем тестовую и тренировочную выборки
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Применяем TF-IDF векторизацию
    current_vectorizer = vectorize() 
    
    # Pipeline позволяет объединить тир шага: 
    # find set of features, generate new features, select only some good features
    model = Pipeline([('tfidf', current_vectorizer), ('clf', MultinomialNB())])
    model.fit(X_train, y_train)  # обучаем
    y_test_predict = model.predict(X_test)
    
print (evaluate(y_test, y_test_predict))

**Random Forest**

In [ ]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    current_vectorizer = vectorize()
    
    model = Pipeline([('tfidf', current_vectorizer), ('clf', RandomForestClassifier())])
    model.fit(X_train, y_train)  
    y_test_predict = model.predict(X_test)
print(evaluate(y_test, y_test_predict))

**Support Vector Classifier**

In [ ]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    current_vectorizer = vectorize()
    
    model = Pipeline([('tfidf', current_vectorizer), ('clf', SVC())])
    model.fit(X_train, y_train)
    y_test_predict = model.predict(X_test)
print(evaluate(y_test, y_test_predict))

**K-neighbors Classifier**

In [ ]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    current_vectorizer = vectorize()
    
    model = Pipeline([('tfidf', current_vectorizer), ('clf', KNeighborsClassifier(5))])
    model.fit(X_train, y_train)
    y_test_predict = model.predict(X_test)
print(evaluate(y_test, y_test_predict))